# Data Continuation

## Modeling Preparation

We'll start off this process by doing much of the same
importing as from last time.

In [1]:
import gc
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import cv2

#statsmodels
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from nltk.corpus import stopwords

from _code.card_selection import card_sampler, \
            plot_card_trends, card_imager, \
                synthesize_names

from _code.cleaner import preprocess

from _code.viz import showImagesHorizontally, word_plot

from _code import card_selection as FUNC_TESTING

from IPython.display import Image

In [46]:
cards = pd.read_parquet('./data/simplified_cards.parquet')

In [79]:
processed_abilities = preprocess(cards['oracle_text'])
fully_processed_abilities = [abilities.split('\n') for abilities in processed_abilities]

In [80]:
cards['abilities_list'] = fully_processed_abilities

In [83]:
cards.head()[['name','oracle_text','abilities_list','median_foil','median_normal']]

,name,oracle_text,abilities_list,median_foil,median_normal
0,Fury Sliver,All Sliver creatures have double strike.,[sliver creature double strike],3.95,0.380
1,Kor Outfitter,"When CARDNAME enters the battlefield, you may ...",[cardname enters battlefield may attach target...,7.78,0.240
2,Siren Lookout,"Flying\nWhen CARDNAME enters the battlefield, ...","[fly , cardname enters battlefield explores ]",0.23,0.060
3,Web,Enchant creature (Target a creature as you cas...,"[enchant creature , enchant creature get +0/...",NaN,0.640
4,Venerable Knight,"When CARDNAME dies, put a +1/+1 counter on tar...",[cardname die put +1/+1 counter target knight ...,0.28,0.095


=======================================================
EXPLORATORY OPTIONS  
=======================================================

In [72]:
_test = cards.iloc[3]['oracle_text']

In [78]:
tokenizer.tokenize(_test)

['Enchant',
 'creature',
 '',
 '\n',
 'Enchanted',
 'creature',
 'gets',
 '+0/+2',
 'and',
 'has',
 'reach',
 '']

In [20]:

from nltk.tokenize import RegexpTokenizer

In [22]:
tokenizer = RegexpTokenizer(r"(\{?[a-zA-Z]+(?:’[a-z]+)?\}?|[+-]?\d\/[+-]?\d|\{\d\d?\})")
tokenizer.tokenize("this is a test!")

['this', 'is', 'a', 'test']

In [77]:
tokenizer = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?|[+-]?\d\/[+-]?\d|\{\d\d?\}|\{.\s?.?\}|\n)|\(.+?\)")
# make texts lowercase
# texts = [str(phrase).lower() for phrase in texts]
tokenizer.tokenize('this is a test\n (of the system)')

['this', 'is', 'a', 'test', '\n', '']

In [57]:
_collect = cards['oracle_text'].sample(50)
_collect = preprocess(_collect)
_collection = [abilities.split('\n') for abilities in _collect]

In [58]:
_collection

[['target creature gain double strike end turn '],
 ['{t} add {c} ',
  ' {1} {t} sacrifice cardname search library basic forest plain island card put onto battlefield tap shuffle'],
 ['{t} add {w}'],
 ['cardname enters battlefield tapped ', ' {t} add {w} {u}'],
 ['cardname enters battlefield tapped unless control mountain forest ',
  ' {t} add {r} {g}'],
 ['outlast {1} {b} '],
 [''],
 ['fly deathtouch ',
  ' cardname die choose one ',
  ' search library three land card reveal put hand shuffle ',
  ' create x x green spirit creature token x number land control'],
 ['add {b} {b} {b}'],
 ['{2} {g} {u} draw card may put land card hand onto battlefield'],
 ['cardname block'],
 ['split second  ',
  ' choose target card graveyard basic land card search owner graveyard hand library card name card exile player shuffle'],
 ['player sacrifice creature create x x black demon creature token fly x total power creature sacrifice way'],
 ['cardname enters battlefield tapped unless two opponent ',
  ' 